# Particle Neural Network

Imports Three handcrafted files, in addition to the standard numpy and matplotlib moduals.  ParticleNet.py is the neural network, which takes as input 4-vectors generated using ExpodentialDistribution.py and GaussianDistribution.py which produced 3-vector momentum and a mass vector each, where the mass vector is radomly drawn from either an expodential or a gaussian distribution respectivly.  PlotResults contains a plot function, which plots the distribution of the precentages of the Particle Neural Network's predictions for a single particle.

In [1]:
from ExpodentialDistribution import Exp
from GaussianDistribution import Gauss
from ParticleNet import ParticleNet
import numpy as np
from PlotResults import plot
import matplotlib.pyplot as plt
import tensorflow as tfv2
import tensorflow.compat.v1 as tf
from tensorflow.keras import layers
tf.disable_v2_behavior()

params = 4

Instructions for updating:
non-resource variables are not supported in the long term


These are the only constants needed to run this simulation thus far.  'p' the momentum is constant for all particles.  Ntrain is the number of samples the particle neural network will train on, and Ntest is the number of particles the neural network will test on.

In [2]:
p = 100
Ntrain = 5
Ntest = 5

Below, we generate our data to feed into the particle neural network.  First we create training and testing data sets from the expodential distribution modual and then the gaussian modual.  Both moduals generate a 4-vector, which includes a 3-vector momentum and a mass vector where the masses are drawn from the respective distribution at random.  Besides the 4-vector, a vector idicating that which distribution each piece of data belongs to is returned, a vector of all zeros for the expodential distribution and a vector of all ones for the gaussian.

In [3]:
ETrain,ETrainVals = Exp(p,Ntrain)
ETest,ETestVals = Exp(p,Ntest)

GTrain,GTrainVals = Gauss(p,Ntrain)
GTest,GTestVals = Gauss(p,Ntest)

Here, we are just concatinating the data from the expodential and gaussian distributions into a single training and testing data set.

In [4]:
Train = np.zeros((2*Ntrain,params+1))
Test = np.zeros((2*Ntest,params+1))

Train[:Ntrain,0:params] = ETrain
Train[Ntrain:,0:params] = GTrain
Train[:Ntrain,params] = ETrainVals
Train[Ntrain:,params] = GTrainVals

Test[:Ntest,0:params] = ETest
Test[Ntest:,0:params] = GTest
Test[:Ntrain,params] = ETrainVals
Test[Ntrain:,params] = GTrainVals

BUFFER_SIZE = Ntrain*2
BATCH_SIZE = params+1
train_dataset = tf.data.Dataset.from_tensor_slices(Train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices(Test).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

Noise = np.random.uniform(0,1,[Ntrain*2,params])


In [5]:
for i in train_dataset.take(-1):
    train = i[0:,0:params]
    train_vals = i[0:,params]
for i in test_dataset.take(-1):
    test = i[0:,0:params]
    test_vals = i[0:,params]
train = np.array(train)
train_vals = np.array(train_vals)
test = np.array(test)
test_vals = np.array(test_vals)

RuntimeError: __iter__() is only supported inside of tf.function or when eager execution is enabled.

In [6]:
def get_y(x):
    return 10 + x*x


def sample_data(n=10000, scale=100):
    data = []

    x = scale*(np.random.random_sample((n,))-0.5)

    for i in range(n):
        yi = get_y(x[i])
        data.append([x[i], yi])

    return np.array(data)

In [7]:
def generator(Z,hsize=[16, 16],reuse=False):
    with tf.variable_scope("GAN/Generator",reuse=reuse):
        h1 = tf.layers.dense(Z,hsize[0],activation=tf.nn.leaky_relu)
        h2 = tf.layers.dense(h1,hsize[1],activation=tf.nn.leaky_relu)
        out = tf.layers.dense(h2,2)

    return out

In [8]:
def discriminator(X,hsize=[16, 16],reuse=False):
    with tf.variable_scope("GAN/Discriminator",reuse=reuse):
        h1 = tf.layers.dense(X,hsize[0],activation=tf.nn.leaky_relu)
        h2 = tf.layers.dense(h1,hsize[1],activation=tf.nn.leaky_relu)
        h3 = tf.layers.dense(h2,2)
        out = tf.layers.dense(h3,1)

    return out, h3

In [9]:
X = tf.placeholder(tf.float32,[None,2])
Z = tf.placeholder(tf.float32,[None,2])

In [10]:
G_sample = generator(X)
r_logits, r_rep = discriminator(Z)
f_logits, g_rep = discriminator(G_sample,reuse=True)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [11]:
disc_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=r_logits,labels=tf.ones_like(r_logits)) + tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,labels=tf.zeros_like(f_logits)))
gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,labels=tf.ones_like(f_logits)))


In [12]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [13]:
def loss():
    gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Generator")
    return gen_vars
gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Generator")
disc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Discriminator")



gen_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(gen_vars[0],var_list = gen_vars) # G Train step
disc_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(disc_vars[0],var_list = disc_vars) # D Train step

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
batch_size=30
sess = tf.Session()
for i in range(100001):
    X_batch = sample_data(n=batch_size)
    Z_batch = sample_Z(batch_size, 2)
    _, dloss = sess.run([disc_step, disc_loss], feed_dict={X: X_batch, Z: Z_batch})
    _, gloss = sess.run([gen_step, gen_loss], feed_dict={Z: Z_batch})

    print ("Iterations: %d\t Discriminator loss: %.4f\t Generator loss: %.4f"%(i,dloss,gloss))


FailedPreconditionError: Attempting to use uninitialized value GAN/Discriminator/dense/bias
	 [[node GAN/Discriminator/dense/bias/read (defined at <ipython-input-8-92f6b187d5a5>:3) ]]

Original stack trace for 'GAN/Discriminator/dense/bias/read':
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 612, in start
 

In [ ]:
gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Generator")
print(gen_vars)
type(gen_vars[0])

Below is the particle neural network being run on the data generated.

In [15]:
predict = ParticleNet(train,train_vals,test)

NameError: name 'train' is not defined

Below, we take a look at the masses for the expodential distribution.

In [ ]:
n, bins, patches = plt.hist(x=ETrain[:,3], bins='auto')
plt.xlabel('Masses Expodential')
plt.ylabel('Frequency')
plt.title('Expodential Distrubution')

Below is a plot of the masses randomly generated from a Gaussian distribution.

In [ ]:
n, bins, patches = plt.hist(x=GTrain[:,3], bins='auto')
plt.xlabel('Masses Normal')
plt.ylabel('Frequency')
plt.title('Normal Distrubution')

In [ ]:
n2, bins2, patches2 = plt.hist(x=ETrain[:,3], bins=np.arange(0,1000,5))
n, bins, patches = plt.hist(x=GTrain[:,3], bins=np.arange(0,1000,5))
plt.xlim(0,600)
plt.ylabel('Frequency')
plt.title('Normal Distrubution')

In [ ]:
integN = np.sum(n2)
integG = np.sum(n)
print(integN)
print(integG)
xmin = np.argwhere(bins==25).flatten()[0]
xmax = np.argwhere(bins==75).flatten()[0]
ratio = np.sum(n2[xmin:xmax])/np.sum(n[xmin:xmax])
print(ratio)

In [ ]:
FalseExps = sum(predict[:Ntest])/Ntest*100
print("The model, on average, thought a Expodential distribution to be just that at " + str(FalseExps[0]) + " percent.")
print("And thought a Expodential distribution to be a Gaussian distribution at " + str(FalseExps[1]) + " percent.")

In [ ]:
FalseGauss = (Ntest-sum(predict[Ntest:]))/Ntest*100
print("The model, on average, thought a Gaussian distribution to be just that at " + str(FalseGauss[0]) + " percent.")
print("And thought a Gaussian distribution to be a Expodential distribution at " + str(FalseGauss[1]) + " percent.")

In [ ]:
dist = Train[:,3]*predict[:,0]
plt.hist(x=Train[:,3],weights=predict[:,1],alpha=0.5, bins=np.arange(0,1000,5))
plt.hist(x=Train[:,3],weights=predict[:,0],alpha=0.5, bins=np.arange(0,1000,5))
plt.xlim(0,200)

In [ ]:
n2, bins2, patches2 = plt.hist(x=ETrain[:,3], bins=np.arange(0,1000,5))
n, bins, patches = plt.hist(x=GTrain[:,3], bins=np.arange(0,1000,5))
plt.xlim(0,600)
plt.ylabel('Frequency')

In [ ]:
def make_generator_model(params,N):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, use_bias=False, input_shape=(4,1)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())    
    
    model.add(layers.Reshape((params, 256,1)))
    assert model.output_shape == (None, params, 256,1)
    
#    for i in range(params-3):
#        x = (params*256)/((i+1)*2)
    model.add(layers.Conv2DTranspose(128, 1, strides=1, padding='same', use_bias=False))
    assert model.output_shape == (None, params, 128,1)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(1, 1, strides=1, padding='same', use_bias=False))
    assert model.output_shape == (None, params, 1, 1)
    
    return model

In [ ]:
generator = make_generator_model(params,Ntrain)

In [ ]:
noise = tf.random.uniform([params])
generated_image = generator(noise, training=False)

In [ ]:
tf.compat.v1.disable_v2_behavior()
tf.compat.v1.enable_eager_execution()